# Imports


In [1]:
import minari
import torch

In [2]:
dataset = minari.load_dataset("D4RL/pointmaze/large-v2")

# Prior Functionality

### Getting functions/methods from other classes

In [3]:
class TrajectorySet: 
    def __init__(self): 
        """
        trajectories: a dictionary housing all of the trajectories. The dictionary structure is: 
            {
                1: [trajectory, length of trajectory]
                2: [ ... ]
                etc...
            } 

        num_trajectories: the number of trajectories currently in the set. 
        """

        self.trajectories = {} 
        self.num_trajectories = 0 
    
    def add_trajectory(self, trajectory):
        self.trajectories[self.num_trajectories] = [trajectory, len(trajectory)] 
        self.num_trajectories += 1
    
    def get_num_trajectories(self):
        return self.num_trajectories

    def get_trajectory(self, index): 
        assert index < self.num_trajectories, "Specified index is too large."
        return self.trajectories[index]
    
    def get_trajectory_set(self): 
        return self.trajectories
    
    def get_total_states(self): 
        sum = 0
        for _, v in self.trajectories.items(): 
            sum += v[1]
        return sum 

    def generate_trajectories(self, n_trajectories: int = 2): 
        """
        Generates a specified number of trajectories and saves them into the TrajectorySet class. 

        This runs the scripted agent, where the agent uses a PD controller to follow a 
        path of waypoints generated with QIteration until it reaches the goal.
        """
        ep_data = dataset.sample_episodes(n_episodes=n_trajectories) # sample trajectories

        # adds all of the sampled trajectories into the TrajectorySet 
        for i in range(len(ep_data)):
            ep = ep_data[i] 

            # Note: only saving states since we only need state representations in the encoder 
            self.add_trajectory(ep.observations["observation"]) 

# Sampling 

### Need to sample and get positive and negative pairs (states). 
### Check out Torch distributions page: https://docs.pytorch.org/docs/stable/distributions.html

In [64]:
def sample_anchor_state(t: list) -> tuple[list, int]: 
    """
    Given a trajectory, we sample the anchor state s_i uniformly. 

    Args: 
        t: The given trajectory we sample from. 

    Returns: 
        A tuple containing [s_i, idx]
        s_i: The state that is sampled, represented as a list of (x,y) coordinates and velocities. 
        idx: The time step of s_i. 
    """
    idx = torch.randint(low=0, high=len(t), size=(1,)).item()
    s_i = t[idx] 
    return [s_i, idx]

def sample_positive_pair_gaussian(t: list, anchor_state: tuple[list, int]) -> tuple[list, int]: 
    """
    Given the same trajectory that s_i was sampled from, 
    center a gaussian distribution around s_i to get obtain its positve pair: s_j. 
    
    Reference: https://docs.pytorch.org/docs/stable/generated/torch.normal.html

    Args: 
        t: The given trajectory, which must be the same as the trajectory that was used to sample the anchor state. 
        anchor_state: The anchor state; a tuple containing [s_i, idx].
        s_i: The state itself.
        idx: The time step of s_i.
    
    Return: 
        A tuple containing [s_j, idx]
        s_j: The state that is sampled, represented as a list of (x,y) coordinates and velocities. 
        idx: The time step of s_j.    
    """
    std = 15  # we use 15 to replicate the paper's hyperparams

    _, si_idx = anchor_state

    sj_idx = len(t) + 1
    while sj_idx >= len(t): # ensures we don't choose an index out of range
        sj_idx = torch.normal(mean=si_idx, std=std, size=(1,))
        sj_idx = int(sj_idx) 
    
    s_j = t[sj_idx] 

    return [s_j, sj_idx]

In [65]:
"""
TESTING CELL!
"""
T = TrajectorySet() 
T.generate_trajectories(n_trajectories=1) 
t = T.get_trajectory(0)[0] 
anchor_state = sample_anchor_state(t)
print(f"s_i: {anchor_state[0]}, at time step: {anchor_state[1]}")

positive_pair = sample_positive_pair_gaussian(t=t, anchor_state=anchor_state)
print(f"s_j: {positive_pair[0]}, at time step: {positive_pair[1]}")


s_i: [-0.40379425  1.01396695  0.06817597 -0.03889898], at time step: 15
s_j: [-0.60063626  0.99893879  3.21868992  0.00358602], at time step: 2
